In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 0llheaven/Llama-3.2-11B-Vision-Radiology-mini
https://huggingface.co/0llheaven/Llama-3.2-11B-Vision-Radiology-mini

In [2]:
!pip install unsloth
!pip install torch

In [3]:
from unsloth import FastVisionModel
from PIL import Image
import numpy as np
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-07-17 10:17:38.196949: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752747458.225167     167 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752747458.233709     167 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
device

device(type='cuda')

In [6]:
model, tokenizer = FastVisionModel.from_pretrained(
    "0llheaven/Llama-3.2-11B-Vision-Radiology-mini",
    load_in_4bit=True,
    use_gradient_checkpointing="unsloth",
)

FastVisionModel.for_inference(model)



==((====))==  Unsloth 2025.7.3: Fast Mllama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

MllamaForConditionalGeneration(
  (model): MllamaModel(
    (vision_model): MllamaVisionModel(
      (patch_embedding): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), padding=valid, bias=False)
      (gated_positional_embedding): MllamaPrecomputedPositionEmbedding(
        (tile_embedding): Embedding(9, 8197120)
      )
      (pre_tile_positional_embedding): MllamaPrecomputedAspectRatioEmbedding(
        (embedding): Embedding(9, 5120)
      )
      (post_tile_positional_embedding): MllamaPrecomputedAspectRatioEmbedding(
        (embedding): Embedding(9, 5120)
      )
      (layernorm_pre): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (layernorm_post): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (transformer): MllamaVisionEncoder(
        (layers): ModuleList(
          (0-31): 32 x MllamaVisionEncoderLayer(
            (self_attn): MllamaVisionAttention(
              (q_proj): Linear4bit(in_features=1280, out_features=1280, bias=False)
       

In [14]:
def predict_radiology_description(image, instruction):
    try:
        messages = [{"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": instruction}
        ]}]
        input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

        inputs = tokenizer(
            image,
            input_text,
            add_special_tokens=False,
            return_tensors="pt",
        ).to(device)

        output_ids = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=1.5,
            min_p=0.1
        )

        generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        return generated_text.replace("assistant", "\n\nassistant").strip()
    except Exception as e:
        return f"Error: {str(e)}"

In [8]:
image_path = '/kaggle/input/xray-image/broken-hand.jpg'
instruction = 'are there fractures present?'

image = Image.open(image_path).convert("RGB")
output = predict_radiology_description(image, instruction)
print(output)


user

are there fractures present?

assistant

Yes, there are fractures present. The image shows a broken arm with two visible fractures, one in the radius bone (the long bone in the forearm) and one in the ulna bone (the shorter bone in the forearm).


In [9]:
image_path = '/kaggle/input/xray-image/cancer-xray.jpg'
instruction = 'is there any presence of cancer or tumors? what is the diagnosis?'

image = Image.open(image_path).convert("RGB")
output = predict_radiology_description(image, instruction)
print(output)


user

is there any presence of cancer or tumors? what is the diagnosis?

assistant

There is a visible mass in the right upper lobe of the lung on this X-ray. The presence of a mass in the lung, as shown in this image, may indicate lung cancer or other serious health complications, including infection and other pulmonary conditions.

As lung cancer and tumors can be life threatening, it is crucial to follow up with a healthcare provider to confirm the diagnosis and obtain further imaging and medical care, including treatment.


In [15]:
image_path = '/kaggle/input/xray-image/broken-leg.jpg'
instruction = 'what is the diagnosis?'

image = Image.open(image_path).convert("RGB")
output = predict_radiology_description(image, instruction)
print(output)

user

what is the diagnosis?

assistant

This X-ray image shows two broken legs, both of which are compound fractures, one of the right femur and one of the right tibia. The fracture of the right femur appears to have some degree of movement between the pieces, indicating potential dislocation and soft tissue damage. Additionally, a possible bone spur or old fracture line can be observed in the lower part of the femur, suggesting a possible history of trauma or a previous fracture. The tibia appears to have more movement between the bone fragments and shows signs of possible soft tissue injury around the right foot as well. These observations imply that this person would likely be hospitalized with possible surgery, an orthopedic specialist would need to further assess these fractures before forming a definitive diagnosis.
